In [1]:
import pickle
import json
import numpy as np
import pandas as pd

# Read Instances

In [2]:
J_list = [100] # personas
M_list = [50] # mesas
G_list = [2,3,4] # grupos
I_list = [2,3,5,10] # candidatos 
lambda_list = [0.5]
seed_list = [i+1 for i in range(20)]
# seed_list = [i+1 for i in range(20)]

instances = []
n_instances = len(J_list)*len(M_list)*len(G_list)*len(I_list)*len(seed_list)
EM_method_names = ["full", "simulate_100", "simulate_1000", "cdf", "pdf", "mult"]



In [3]:
df_list = []
for J in J_list:
    for M in M_list:
        for G in G_list:
            for I in I_list:
                for lambda_ in lambda_list:
                        for method in EM_method_names:
                            for seed in seed_list:
                                pickle_path = f'results/J{J}_M{M}_G{G}_I{I}_lambda{int(100*lambda_)}/{method}_cv1000/{seed}.pickle'
                                # check if path exists
                                try:
                                    with open(pickle_path, 'rb') as f:
                                        data = pickle.load(f)
                                    p = data['p']
                                    p_est = data['p_est']
                                    mean_error = np.mean(np.abs(p-p_est))
                                    max_error = np.max(np.abs(p-p_est))
                                    time = data['time']
                                    # print(J,M,G,I,lambda_,method,seed)
                                    if (J == 200) & (M == 50) & (G == 4) & (I == 2) & (lambda_ == 0.5) & (seed == 12):
                                        print(method, mean_error)
                                        # break
                                    if 'simulate' in method:
                                        simulate_time = data['simulation_time']
                                    else:
                                        simulate_time = np.nan
                                    iter = data['iterations']
                                    end = data['end']
                                    df_list.append([J,M,G,I,lambda_,method,seed,time,simulate_time,iter,end,mean_error,max_error])
                                except:
                                    print(f'Error in {pickle_path}')
                                    continue

# pd.DataFrame(df_list, columns=['J','M','G','I','lambda','method','seed','time','simulate_time','iter','end','mean_error','max_error']).to_csv('results/results.csv')
df = pd.DataFrame(df_list, columns=['J','M','G','I','lambda','method','seed','time','simulate_time','iter','end','mean_error','max_error'])                                  

In [105]:
df[(df['method'] == 'simulate_1000') & (df['I'] == 10)]['simulate_time'].min()

472.984171628952

In [40]:
# open pickle
# with open('results/J200_M50_G4_I2_lambda50/cdf/2.pickle', 'rb') as handle:
#     data = pickle.load(handle)
# print(data['mean error'])
# print(data['p'])
# # print('----')
# with open('results/J200_M50_G4_I2_lambda50/cdf/12_v2.pickle', 'rb') as handle:
#     data = pickle.load(handle)
# print(data['p_est'])
# print(data['p'])

In [41]:
# grab today date

# df.to_csv('results_28_07_2023.csv')

# Make Tables

In [16]:
# rows where error is nan

# df_nan = df[df['max_error'].isna()]
# df_nan

,J,M,G,I,lambda,method,seed,time,simulate_time,iter,end,mean_error,max_error


In [42]:
# df[df['end'] == 2]['method'].unique()

array(['cdf', 'pdf', 'simulate_100', 'simulate_1000', 'simulate_500'],
      dtype=object)

In [37]:
# def transform_seconds_to_time(seconds):
#     if seconds < 1:
#         return f'{np.round(seconds*1000)} ms'
#     if seconds > 60:
#         return f'{np.round(seconds/60)} min'
#     else:
#         return f'{np.round(seconds)} sec'


In [4]:
# choose method order
method_order = ['full', 'simulate_100', 'simulate_1000', 'cdf', 'pdf', 'mult']

In [63]:
# Check what happens when it does not end

# df_time = df.groupby(['J','M','G','I','end','method']).mean().reset_index().pivot(index=['J','M','G','I', 'end'], columns='method', values='time').reset_index()
# df_time

## Time


In [5]:
# replace time for nan if end = -1 
df.loc[df['end'] == -1, 'time'] = np.nan
df['total_time'] = df['time'] + df['simulate_time'].fillna(0)

In [6]:
# grab the average time with J,M,G,I as row and method as column
df_time = df.groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values='total_time').reset_index()
df_time[['J', 'M', 'G', 'I']] = df_time[['J', 'M', 'G', 'I']].astype(int)

# round to 4 decimals for methods
df_time[method_order] = df_time[method_order].round(3)
df_time[method_order] = df_time[method_order].applymap(lambda x: "{:.3f}".format(x))

# add separator for thousands and put between brackets for those that have a comma
df_time[method_order] = df_time[method_order].applymap(lambda x: f'{float(x):,.3f}' if float(x) >= 1000 else x) #.replace(',', ' ')

# display floats with 3 decimals even if they are 0'seed

# put numbers with commas between brackets
# df_time[method_order] = df_time[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 'nan' with '-' 
df_time = df_time.replace('nan', '{-}')



# reorder columns
J = 100
df_time_latex = pd.DataFrame()
# change order of columns and use english notation
df_time_latex[['C', 'G'] + method_order] = df_time.loc[df_time['J'] == J, ['I', 'G'] + method_order]
# sort by C and G
df_time_latex = df_time_latex.sort_values(['C', 'G'])


df_time_latex.loc[df_time_latex['C'].duplicated(),['C']] = ''

# add empty columns between 'C' and methods
df_time_latex.insert(2, 'empty1', '')

# transform to latex
print(df_time_latex.to_latex(index=False, escape=False, column_format='cccccccccc', float_format=":.%4f"))

# # replace I by C
# df_time['C'] = df_time['I']

\begin{tabular}{cccccccccc}
\toprule
C & G & empty1 & full & simulate_100 & simulate_1000 & cdf & pdf & mult \\
\midrule
2 & 2 &  & 15.478 & 74.050 & 731.258 & 1.093 & 0.151 & 0.003 \\
 & 3 &  & 31.077 & 73.251 & 718.892 & 1.988 & 0.191 & 0.004 \\
 & 4 &  & 45.786 & 71.464 & 708.081 & 3.037 & 0.216 & 0.005 \\
3 & 2 &  & 555.319 & 72.790 & 716.579 & 4.608 & 0.188 & 0.004 \\
 & 3 &  & 2,091.768 & 69.354 & 705.523 & 8.166 & 0.241 & 0.005 \\
 & 4 &  & 3,029.651 & 66.477 & 665.910 & 11.256 & 0.256 & 0.005 \\
5 & 2 &  & {-} & 67.697 & 680.884 & 13.982 & 0.170 & 0.004 \\
 & 3 &  & {-} & 62.270 & 624.810 & 23.177 & 0.210 & 0.004 \\
 & 4 &  & {-} & 58.257 & 585.061 & 34.268 & 0.256 & 0.005 \\
10 & 2 &  & {-} & 59.587 & 601.103 & 49.965 & 0.165 & 0.003 \\
 & 3 &  & {-} & 54.035 & 538.743 & 98.016 & 0.225 & 0.005 \\
 & 4 &  & {-} & 50.622 & 502.069 & 124.858 & 0.264 & 0.005 \\
\bottomrule
\end{tabular}



Simulation time


In [20]:
# method_order_sim = ['simulate_timesimulate_100', 'timesimulate_100', 'simulate_timesimulate_1000', 'timesimulate_1000']
method_order_sim = ['simulate_timesimulate_100', 'timesimulate_100', 'simulate_timesimulate_1000', 'timesimulate_1000']


# grab the average time with J,M,G,I as row and method as column
df_sim = df.loc[df['method'].str.contains('simulate_1')].groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values=['time', 'simulate_time']).reset_index()

df_sim[['J', 'M', 'G', 'I']] = df_sim[['J', 'M', 'G', 'I']].astype(int)
df_sim.columns = [f'{col[0]}{col[1]}' for col in df_sim.columns]

# round to 4 decimals for methods
df_sim[method_order_sim] = df_sim[method_order_sim].round(3)
df_sim[method_order_sim] = df_sim[method_order_sim].applymap(lambda x: "{:.3f}".format(x))

# add separator for thousands and put between brackets for those that have a comma
df_sim[method_order_sim] = df_sim[method_order_sim].applymap(lambda x: f'{float(x):,.3f}' if float(x) >= 1000 else x) #.replace(',', ' ')

# display floats with 3 decimals even if they are 0'seed

# put numbers with commas between brackets
# df_sim[method_order] = df_sim[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 'nan' with '-' 
df_sim = df_sim.replace('nan', '{-}')

# reorder columns
J = 100
df_time_sim_latex = pd.DataFrame()

# change order of columns and use english notation
df_time_sim_latex[['C', 'G'] + method_order_sim] = df_sim.loc[df_sim['J'] == J, ['I', 'G'] + method_order_sim]
# sort by C and G
df_time_sim_latex = df_time_sim_latex.sort_values(['C', 'G'])


df_time_sim_latex.loc[df_time_sim_latex['C'].duplicated(),['C']] = ''

# add empty columns between 'C' and methods
df_time_sim_latex.insert(2, 'empty1', '')

# transform to latex
print(df_time_sim_latex.to_latex(index=False, escape=False, float_format=":.%4f"))


\begin{tabular}{lrlllll}
\toprule
C & G & empty1 & simulate_timesimulate_100 & timesimulate_100 & simulate_timesimulate_1000 & timesimulate_1000 \\
\midrule
2 & 2 &  & 73.409 & 0.820 & 740.046 & 0.919 \\
 & 3 &  & 72.915 & 2.295 & 722.202 & 14.155 \\
 & 4 &  & 72.732 & 2.522 & 686.560 & 12.015 \\
3 & 2 &  & 71.105 & 1.855 & 733.173 & 12.359 \\
 & 3 &  & 68.670 & 1.881 & 699.799 & 18.290 \\
 & 4 &  & 69.916 & 2.331 & 703.638 & 24.347 \\
5 & 2 &  & 69.471 & 1.627 & 693.218 & 18.157 \\
 & 3 &  & 65.756 & 2.196 & 567.549 & 9.111 \\
 & 4 &  & 64.044 & 1.929 & 652.590 & 23.535 \\
10 & 2 &  & 65.161 & 1.515 & 655.398 & 15.595 \\
 & 3 &  & 59.866 & 1.474 & 525.634 & 8.404 \\
 & 4 &  & 57.256 & 1.479 & 578.101 & 18.743 \\
\bottomrule
\end{tabular}



## Error

In [9]:
df.loc[df['end'] == -1,'mean_error'] = np.nan

In [10]:
# CG   

# grab the average time with J,M,G,I as row and method as column
df_error = df.groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values='mean_error').reset_index()
df_error[['J', 'M', 'G', 'I']] = df_error[['J', 'M', 'G', 'I']].astype(int)
# 
# round to 4 decimals for methods
df_error[method_order] = df_error[method_order].round(3)
df_error[method_order] = df_error[method_order].applymap(lambda x: "{:.3f}".format(x))

# add separator for thousands and put between brackets for those that have a comma
df_error[method_order] = df_error[method_order].applymap(lambda x: f'{float(x):,.3f}' if float(x) >= 1000 else x) #.replace(',', ' ')

# display floats with 3 decimals even if they are 0'seed

# put numbers with commas between brackets
# df_error[method_order] = df_error[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 'nan' with '-' 
df_error = df_error.replace('nan', '{-}')



# reorder columns
J = 100
df_error_latex = pd.DataFrame()
# change order of columns and use english notation
df_error_latex[['C', 'G'] + method_order] = df_error.loc[df_error['J'] == J, ['I', 'G'] + method_order]
# sort by C and G
df_error_latex = df_error_latex.sort_values(['C', 'G'])


df_error_latex.loc[df_error_latex['C'].duplicated(),['C']] = ''

# add empty columns between 'C' and methods
df_error_latex.insert(2, 'empty1', '')

# transform to latex
print(df_error_latex.to_latex(index=False, escape=False, column_format='cccccccccc', float_format=":.%4f"))

# # replace I by C
# df_error['C'] = df_time['I']

\begin{tabular}{cccccccccc}
\toprule
C & G & empty1 & full & simulate_100 & simulate_1000 & cdf & pdf & mult \\
\midrule
2 & 2 &  & 0.012 & 0.012 & 0.012 & 0.013 & 0.014 & 0.011 \\
 & 3 &  & 0.015 & 0.019 & 0.015 & 0.016 & 0.016 & 0.015 \\
 & 4 &  & 0.018 & 0.031 & 0.019 & 0.019 & 0.019 & 0.018 \\
3 & 2 &  & 0.011 & 0.012 & 0.011 & 0.013 & 0.011 & 0.010 \\
 & 3 &  & 0.014 & 0.032 & 0.016 & 0.016 & 0.015 & 0.014 \\
 & 4 &  & 0.018 & 0.048 & 0.026 & 0.020 & 0.018 & 0.018 \\
5 & 2 &  & {-} & 0.014 & 0.009 & 0.010 & 0.009 & 0.008 \\
 & 3 &  & {-} & 0.034 & 0.020 & 0.015 & 0.014 & 0.014 \\
 & 4 &  & {-} & 0.042 & 0.031 & 0.015 & 0.014 & 0.014 \\
10 & 2 &  & {-} & 0.016 & 0.009 & 0.009 & 0.008 & 0.006 \\
 & 3 &  & {-} & 0.026 & 0.022 & 0.010 & 0.009 & 0.009 \\
 & 4 &  & {-} & 0.032 & 0.027 & 0.013 & 0.013 & 0.012 \\
\bottomrule
\end{tabular}



In [53]:
# CG   

# grab the average time with J,M,G,I as row and method as column
df_error = df.groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values='mean_error').reset_index()
df_error[['J', 'M', 'G', 'I']] = df_error[['J', 'M', 'G', 'I']].astype(int)
# 
# round to 4 decimals for methods
df_error[method_order] = df_error[method_order].round(3)
df_error[method_order] = df_error[method_order].applymap(lambda x: "{:.3f}".format(x))

# add separator for thousands and put between brackets for those that have a comma
df_error[method_order] = df_error[method_order].applymap(lambda x: f'{float(x):,.3f}' if float(x) >= 1000 else x) #.replace(',', ' ')

# display floats with 3 decimals even if they are 0'seed

# put numbers with commas between brackets
# df_error[method_order] = df_error[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 'nan' with '-' 
df_error = df_error.replace('nan', '{-}')



# reorder columns
J = 100
df_error_latex = pd.DataFrame()
# change order of columns and use english notation
df_error_latex[['G', 'C'] + method_order] = df_error.loc[df_error['J'] == 100, ['G', 'I'] + method_order]
# sort by C and G
df_error_latex = df_error_latex.sort_values(['G', 'C'])


df_error_latex.loc[df_error_latex['G'].duplicated(),['G']] = ''

# add empty columns between 'C' and methods
df_error_latex.insert(2, 'empty1', '')

# transform to latex
print(df_error_latex.to_latex(index=False, escape=False, column_format='cccccccccc', float_format=":.%4f"))

# # replace I by C
# df_error['C'] = df_time['I']

\begin{tabular}{cccccccccc}
\toprule
G & C & empty1 & full & simulate_1000 & simulate_100 & cdf & pdf & mult \\
\midrule
2 & 2 &  & 0.011 & 0.011 & 0.011 & 0.011 & 0.011 & 0.011 \\
 & 3 &  & 0.010 & 0.010 & 0.011 & 0.010 & 0.010 & 0.010 \\
 & 5 &  & {-} & 0.009 & 0.012 & 0.008 & 0.009 & 0.009 \\
 & 10 &  & {-} & 0.007 & 0.012 & 0.006 & 0.007 & 0.006 \\
3 & 2 &  & 0.014 & 0.014 & 0.016 & 0.014 & 0.014 & 0.015 \\
 & 3 &  & 0.014 & 0.015 & 0.025 & 0.014 & 0.014 & 0.015 \\
 & 5 &  & {-} & 0.016 & 0.025 & 0.013 & 0.013 & 0.013 \\
 & 10 &  & {-} & 0.015 & 0.020 & 0.009 & 0.009 & 0.009 \\
4 & 2 &  & 0.017 & 0.018 & 0.025 & 0.017 & 0.017 & 0.018 \\
 & 3 &  & 0.017 & 0.020 & 0.034 & 0.017 & 0.017 & 0.017 \\
 & 5 &  & {-} & 0.022 & 0.032 & 0.014 & 0.014 & 0.014 \\
 & 10 &  & {-} & 0.020 & 0.024 & 0.012 & 0.012 & 0.012 \\
\bottomrule
\end{tabular}



old


In [7]:
# grab the average time with J,M,G,I as row and method as column
df_time = df.groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values='time').reset_index()
df_time[['J', 'M', 'G', 'I']] = df_time[['J', 'M', 'G', 'I']].astype(int)



# reorder columns
df_time = df_time[['J','M','G','I'] + method_order]

# apply transform_seconds_to_time to all columns except J,M,G,I
# df_time[method_order] = df_time[method_order].applymap(transform_seconds_to_time)

# filter only instances to use in order
# df_time = df_time[df_time.apply(lambda x: (x['J'],x['M'],x['G'],x['I']) in instances_to_use, axis=1)]

# get instance in only one column as a tuple with English notation J:I M:B G:G I:C
df_time['(I,G,C,B)'] = df_time.apply(lambda x: f'$({int(x[0])},{int(x[2])},{int(x[3])},{int(x[1])})$', axis=1)

df_time = df_time[['(I,G,C,B)'] + method_order]

# round to 4 decimals for methods
df_time[method_order] = df_time[method_order].round(3)

# add separator for thousands and put between brackets for those that have a comma
df_time[method_order] = df_time[method_order].applymap(lambda x: f'{x:,}') #.replace(',', ' ')

# put numbers with commas between brackets
df_time[method_order] = df_time[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 'nan' with '-' 
df_time = df_time.replace('nan', '{-}')



# print to latex with the rounded decimals aligned by dot
print(df_time.to_latex(index=False, escape=False, column_format='lccccccc', float_format="%.4f"))

\begin{tabular}{lccccccc}
\toprule
(I,G,C,B) & full & simulate_1000 & simulate_100 & cdf & pdf & mult \\
\midrule
$(100,2,2,50)$ & 14.51 & 0.471 & 0.466 & 0.851 & 0.14 & 0.003 \\
$(100,2,3,50)$ & 549.902 & 4.865 & 1.603 & 3.604 & 0.217 & 0.003 \\
$(100,2,5,50)$ & {-} & 17.752 & 1.882 & 12.315 & 0.173 & 0.003 \\
$(100,2,10,50)$ & {-} & 15.861 & 1.624 & 61.702 & 0.181 & 0.003 \\
$(100,3,2,50)$ & 34.235 & 6.427 & 1.9 & 1.99 & 0.208 & 0.004 \\
$(100,3,3,50)$ & {2,115.5} & 24.081 & 2.019 & 11.373 & 0.265 & 0.004 \\
$(100,3,5,50)$ & {-} & 23.091 & 1.801 & 41.609 & 0.279 & 0.005 \\
$(100,3,10,50)$ & {-} & 18.532 & 1.612 & 210.639 & 0.221 & 0.004 \\
$(100,4,2,50)$ & 51.563 & 20.827 & 2.408 & 3.421 & 0.233 & 0.003 \\
$(100,4,3,50)$ & {2,808.304} & 25.409 & 2.515 & 15.927 & 0.32 & 0.005 \\
$(100,4,5,50)$ & {-} & 22.102 & 2.047 & 47.176 & 0.281 & 0.004 \\
$(100,4,10,50)$ & {-} & 22.014 & 1.701 & 355.949 & 0.308 & 0.004 \\
$(200,2,2,50)$ & 59.634 & 0.919 & 0.82 & 1.385 & 0.176 & 0.003 \\
$(200,2,3

In [ ]:
# grab the average time with J,M,G,I as row and method as column
df_error = df.groupby(['J','M','G','I','method']).mean().reset_index().pivot(index=['J','M','G','I'], columns='method', values='mean_error').reset_index()
df_error[['J', 'M', 'G', 'I']] = df_error[['J', 'M', 'G', 'I']].astype(int)

# reorder columns
df_error = df_error[['J','M','G','I'] + method_order]

# apply transform_seconds_to_error to all columns except J,M,G,I
# df_error[method_order] = df_error[method_order].applymap(transform_seconds_to_error)

# filter only instances to use in order
df_error = df_error[df_error.apply(lambda x: (x['J'],x['M'],x['G'],x['I']) in instances_to_use, axis=1)]

# get instance in only one column as a tuple with English notation J:I M:B G:G I:C
df_time['(I,G,C,B)'] = df_time.apply(lambda x: f'$({int(x[0])},{int(x[2])},{int(x[3])},{int(x[1])})$', axis=1)

df_error = df_error[['(I,B,G,C)'] + method_order]

# round to 4 decimals for methods
df_error[method_order] = df_error[method_order].round(3)

# add separator for thousands and put between brackets for those that have a comma
df_error[method_order] = df_error[method_order].applymap(lambda x: f'{x:,}') #.replace(',', ' ')

# put numbers with commas between brackets
df_error[method_order] = df_error[method_order].applymap(lambda x: f'{{{x}}}' if ',' in x else x)

# replace 0 with -
df_error = df_error.replace('0.0', '{-}')


# print to latex with the rounded decimals aligned by dot
print(df_time.to_latex(index=False, escape=False, column_format='lccccccc', float_format="%.4f"))

\begin{tabular}{lccccccc}
\toprule
(I,B,G,C) & full & simulate_100 & simulate_1000 & cdf & pdf & mult \\
\midrule
$(100,50,2,2)$ & 0.0103 & 0.0101 & 0.0103 & 0.0543 & 0.0101 & 0.0103 \\
$(100,50,2,10)$ & 0.0452 & 0.0125 & 0.0080 & 0.0070 & 0.0090 & 0.0069 \\
$(100,50,3,3)$ & 0.0134 & 0.0246 & 0.0146 & 0.0135 & 0.0134 & 0.0138 \\
$(100,50,4,5)$ & 0.1038 & 0.0318 & 0.0231 & 0.0138 & 0.0138 & 0.0139 \\
$(200,50,2,2)$ & 0.0075 & 0.0078 & 0.0075 & 0.1087 & 0.0073 & 0.0081 \\
$(200,50,2,10)$ & 0.0452 & 0.0130 & 0.0084 & 0.0043 & 0.0053 & 0.0046 \\
$(200,50,3,3)$ & 0.0685 & 0.0254 & 0.0125 & 0.0087 & 0.0085 & 0.0091 \\
$(200,50,4,5)$ & 0.1038 & 0.0366 & 0.0263 & 0.0110 & 0.0109 & 0.0112 \\
\bottomrule
\end{tabular}

